# RECORD LINKAGE: EXCERCISE AND ASSIGNMENT

----

This notebook will provide you with an instruction into Record Linkage using Python. Upon completion of this notebook you will be able to apply record linkage techniques using the *recordlinkage* package to combine data from different sources in Python. 
It will lead you through all the steps necessary for a sucessful record linkage starting with data preparation  including pre-processing, cleaning and standardization of data.

## Table of Contents
- [The Principles of Record Linkage](#The-Principles-of-Record-Linkage)
- [Linking Patents to Grants](#Linking-Patents-to-Grants)
- [Import of Packages](#Import-of-Packages)
- [Getting Grant and Patent Info](#Getting-Grant-and-Patent-Info)
- [The Importance of Pre-Processing](#The-Importance-of-Pre-Processing)
- [Record Linkage](#Record-Linkage)
- [References and Further Readings](#References-and-Further-Readings)

## The Principles of Record Linkage
The goal of record linkage is to determine if pairs of records describe the same identity. For instance, this is important for removing duplicates from a data source or joining two separate data sources together. Record linkage also goes by the terms data matching, merge/purge, duplication detection, de-duping, reference matching, entity resolution, disambiguation, co-reference/anaphora in various fields.

There are several approaches to record linkage that include 
    - exact matching, 
    - rule-based linking and 
    - probabilistic linking. 
- An example of **exact matching** is joining records based on a direct identifier. This is we have already have done in SQL by joining tables in the last lecture and lab. 
- **Rule-based matching** involves applying a cascading set of rules that reflect the domain knowledge of the - records being linked. 
- In **probabilistic record linkages**, linkage weights are estimated to calculate the probability of a certain match.

In practical applications you will need record linkage techiques to combine information addressing the same entity that is stored in different data sources. Record linkage will also help you to address the quality of different data sources. For example, if one of your databases has missing values you might be able to fill those by finding an identical pair in a different data source. Overall, the main applications of record linkage are
    1. Merging two or more data files 
    2. Identifying the intersection of the two data sets 
    3. Updating data files (with the data row of the other data files) and imputing missing data
    4. Entity disambiguation and de-duplication

## Linking Patents to Grants

In this lab we will link records obtained from the patent data base (http://www.patentsview.org/web/#viz/relationships) to the data on federal grants we have in our database. In both datasets we have names that we can use to link. We have the PI on the grants and inventor names on patents.

## Import of Packages
Python provides us with some tools we can use for record linkages so we don't have to start from scratch and code our own linkage algorithms. So before we start we need to load the package recordlinkage. To fully function this packages uses other packages which also need to be imported. Thus we are adding more packages to the ones you are already familiar with.

In [ ]:
# general use imports
%pylab inline
import datetime
import numpy as np
import os
import six
import warnings
import matplotlib.pyplot as plt
import re

# pandas-related imports
from __future__ import print_function
import pandas as pd
import scipy
import sklearn

# record linkage package
import recordlinkage as rl
from recordlinkage.preprocessing import clean, phonenumbers, phonetic

# CSV file reading-related imports
import csv

# sqlalchemy an psycopg2 are sql connection packages
from sqlalchemy import create_engine


print( "Imports loaded at " + str( datetime.datetime.now() ) )

## Getting Grant and Patent Info
We can get the data we need from our database and save the the required information in a dataframe. We therefore have to connect to the database and run a SQL query, as we did in the last session. This is what you will do in your assignment. In the lab we will prepare the Patent info. 

In [ ]:
# For Inventor Data
inventor=pd.read_csv('~/FederalReporter/inventor.csv',error_bad_lines=False, encoding='utf8')

In [ ]:
# Checking the attributes of the variables in the loaded dataframe
inventor.dtypes

In [ ]:
# Shape of the data frame
inventor.shape

In [ ]:
# Identifying missing values by using the describe function
inventor.head(100)

In [ ]:
# Looking at the names which have the maximum frequencies associated with them
inventor['name_last'].value_counts()

## The Importance of Pre-Processing
Data pre-processing is an important step in a data anlysis project in general, in record linkage applications in particular. The goal of pre-processing is to transform messy data into a dataset that can be used in a project workflow.

Linking records from different data sources comes with different challenges that need to be addressed by the analyst. The analyst must determine whether or not two entities (individuals, businesses, geographical units) on two different files are the same. This determination is not always easy. In most of the cases there is no common uniquely identifing characteristic for a entity. For example, is Bob Miller from New Yor the same person as Bob Miller from Chicago in a given dataset? This detemination has to be executed carefully because consequences of wrong linkages may be substantial (is person X the same person as the person X on the list of identified terrorists). Pre-processing can help to make better informed decisions.

Pre-processing can be difficult because there are a lot of things to keep in mind. For example, data input errors, such as typos, misspellings, truncation, abbreviations, and missing values need to be corrected. Literature shows that preprocessing can improve matches. In some situations, 90% of the improvement in matching efficiency may be due to preprocessing. The most common reason why matching projects fail is lack of time and resources for data cleaning. 

In the following we will walk you through some pre-processing steps, these include but are not limited to removing spaces, parsing fields, and standardizing strings.

### Parsing String Variables

By default, the split method returns a list of strings obtained by splitting the original string on spaces or commas, etc. The record linkage package comes with a build in cleaning function we can also use. In addition, we can extract information from strings for example by using regex search commands.

### Regular Expressions - regex
When defining a regular expression search pattern, it is a good idea to start out by writing down, explicitly, in plain English, what you are trying to search for and exactly how you identify when you've found a match.
For example, if we look at an author field formatted as "<last_name> , <first_name> <middle_name>", in plain English, this is how I would explain where to find the last name: "starting from the beginning of the line, take all the characters until you see a comma."

We can build a regular expression that captures this idea from the following components:
- ^ Matches beginning of the line
- . Matches any character
- .+ A modifier that means "match one or more of the preceding expression"

In a regular expression, there are special reserved characters and character classes like those in the list above. Anything that is not a special character or class is just looked for explicitly (for example, a comma is not a special character in regular expressions, so if it is in a regular expression pattern, the regular expression processor will just be looking for a comma in the string, at that point in the pattern).

Note: if you want to actually look for one of these reserved characters, it must be escaped, so that, for example, the expression looks for a literal period, rather than the special regular expression meaning of a period. To escape a reserved character in a regular expression, precede it with a back slash ( "." ).
This results in the regular expression: ^.+,

We start at the beginning of the line ( "^" ), matching any characters ( ".+" ) until we come to the literal character of a comma ( "," ).

In python, to use a regular expression like this to search for matches in a given string, we use the built-in "re" package ( https://docs.python.org/2/library/re.html ), specifically the "re.search()" method. To use "re.search()", pass it first the regular expression you want to use to search, enclosed in quotation marks, and then the string you want to search within. 

#### REGEX CHEATSHEET


    - abc...     Letters
    - 123...     Digits
    - \d         Any Digit
    - \D         Any non-Digit Character
    - .          Any Character
    - \.         Period
    - [a,b,c]    Only a, b or c
    - [^a,b,c]   Not a,b, or c
    - [a-z]      Characters a to z
    - [0-9]      Numbers 0 to 9
    - \w any     Alphanumeric chracter
    - \W         any non-Alphanumeric character
    - {m}        m Repetitions
    - {m,n}      m to n repetitions
    - *          Zero or more repetitions
    - +          One or more repetitions
    - ?          Optional Character
    - \s         any Whitespace
    - \S         any non-Whitespace character
    - ^...$      Starts & Ends
    - (...)      Capture Group
    - (a(bc))    Capture sub-Group
    - (.*)       Capture All
    - (abc|def)  Capture abc or def
     
#### EXAMPLES
    - (\d\d|\D) will match 22X, 23G, 56H, etc...
    - \w will match any characters between 0-9 or a-z
    - \w{1-3} will match any alphanumeric character of a length of 1 to 3. 
    - (spell|spells) will match spell or spells

#### Clean Inventor Data
Now we will clean and preprocess the inventor Data. We need to remove whitespaces, make sure that everything is in upper case, we need to parse the first name and the last name, and harmonize all the other information we need for the linkage

In [ ]:
# Remove Whitespaces
inventor.rename(columns=lambda x: x.strip(), inplace=True)

In [ ]:
# Cleaning names (using the record linkage package tool, see imports)
# Clean removes any characters such as '-', '.', '/', '\', ':', brackets of all types. 
inventor['name_last']=clean(inventor['name_last'], lowercase=False, remove_brackets=False)
inventor['name_first']=clean(inventor['name_first'], lowercase=False, remove_brackets=False)

In [ ]:
# Getting the first character in the cleaned name variable
inventor['name_middle'] = inventor.name_first.str.split(' ').str.get(1)
inventor['name_first'] = inventor.name_first.str.split(' ').str.get(0)

In [ ]:
# Upcasing names
inventor['name_last']=inventor.name_last.str.upper()
inventor['name_first']=inventor.name_first.str.upper()
inventor['name_middle']=inventor.name_middle.str.upper()

In [ ]:
inventor.head(100)

Now we are done with the inital data prep work for the inventor file. Please keep in mind that we just provided some examples for you to demonstrate the process. You can add as many further steps to it as necessary. 

#### Clean Grant Data
Now we will clean and preprocess the grants Data. This will be your assignment. We need clean names, zipcode, and a year variable. You can start working on this when we are done with the linkage.

#### YOU CAN INSERT THE CODE HERE OR GENERATE A NEW NOTEBOOK

In [ ]:
# Load Prepared Grant Data
grants=pd.read_csv('~/FederalReporter/grants.csv',error_bad_lines=False, encoding='utf8')

In [ ]:
grants.head()

## Record Linkage
The record linkage package is a quite powerful tool for you to use when you want to link records within a dataset or across multiple datasets. It comes with different bulid in distances metrics and comparison functions, however, it also allows you to create your own. In general record linkage is divided in several steps. 
We've already done the pre-processing. We will add one more thing: a soundex.

In [ ]:
## The phonetic() function is used to convert strings into their corresponding phonetic codes. 
## This is particularly useful when comparing names where different possible spellings make it difficult to find 
## exact matches (Ex. Jillian and Gillian).

grants["phonetic_first"] = phonetic(grants["name_first"], method="nysiis")
grants["phonetic_last"] = phonetic(grants["name_last"], method="nysiis")

inventor["phonetic_first"] = phonetic(inventor["name_first"], method="nysiis")
inventor["phonetic_last"] = phonetic(inventor["name_last"], method="nysiis")

### Indexing

Indexing allows you to create candidate links, which basically means identifying pairs of data rows which might refer to the same real world entity. This is also called the comparison space (matrix). There are different ways to index data. The easiest is to create a full index and consider every pair a match. This is also the least efficient method, because we will be comparing every row of one dataset with every row of the other dataset.

If we had 10,000 records in data frame A and 100,000 records in data frame B, we would have 1,000,000,000 candidate links. You can see that comparing over a full index is getting inefficient when working with big data.

In [ ]:
# Let's generate a full index first (comparison table of all possible linkage combinations)
#indexer = rl.FullIndex()
#pairs = indexer.index(grants, inventor)
# Returns a pandas MultiIndex object
## How many records do we have?
print (len(grants), len(inventor))

We can do better if we actually include our knowledge about the data to eliminate bad link from the start. This can be done through blocking. The recordlinkage packages gives you multiple options for this. For example, you can block by using variables, which menas only links exactly equal on specified values will be kept. You can also use a neighbourhood index in which the rows in your dataframe are ranked by some value and python will only link between the rows that are closeby.

In [ ]:
indexerBL = rl.BlockIndex(on=['name_first', 'name_last'])
pairs2 = indexerBL.index(grants, inventor)
# Returns a pandas MultiIndex object
print(len(pairs2))

### Record Comparison

After you have created a set of candidate links, you’re ready to begin comparing the records associated with each candidate link. In recordlinkage you must initiate a Compare object prior to performing any comparison functionality between records. This object stores both dataframes, the candidate links, and a vector containing comparison results. Further, the Compare object contains the methods for performing comparisons. The code block below initializes the comparison object.

In [ ]:
# Initiate compare object (we are using the blocked ones here)
# You want to give python the name of the MultiIndex and the names of the datasets
compare_cl = rl.Compare()

Currently there are five specific comparison methods within recordlinkage: Compare.exact(), Compare.string(), Compare.numeric(), Compare.geo(), and Compare.date(). The Compare.exact() method is simple: if two values are an exact match a comparison score of 1 is returned, otherwise 0 is retured. The Compare.string() method is a bit more complicated and generates a score based on well known string-comparison algorithms (for this example Levenshtein or Jaro Winkler).

In [ ]:
compare_cl.string('name_first', 'name_first', method='jarowinkler', threshold=0.85, label='name_first')
compare_cl.string('name_last', 'name_last', method='jarowinkler', threshold=0.85, label='name_last')

compare_cl.string('phonetic_first', 'phonetic_first', method='jarowinkler', threshold=0.85, label='phonetic_first')
compare_cl.string('phonetic_last', 'phonetic_last', method='jarowinkler', threshold=0.85, label='phonetic_last')

compare_cl.exact('organization_country', 'country', label='country')
compare_cl.exact('fy', 'fy', label='year')

In [ ]:
## The comparing of record pairs starts when the compute method is called. 
## All attribute comparisons are stored in a DataFrame with horizontally the features and vertically the record pairs.

features = compare_cl.compute(pairs2, grants, inventor)
features.head(200)

### Classification

Now we have to decide which records belong to one person. We can do this pretty simple, but we can also use machine learning methods to classify records.

In [ ]:
## Simple Classification: Check for how many attributes records are identical by summing the comparison results.
features.sum(axis=1).value_counts().sort_index(ascending=False)

In [ ]:
matches = features[features.sum(axis=1) > 4]
print(len(matches))

Now that we have the list of matches we can fuse our dataset, becasue at the end we want to have a combined dataset. We are using a function for this task.

In [ ]:
def fuse(dfA, dfB, dfmatches):
    newDF = dfA.copy()
    columns = dfB.columns.values
    
    for col in columns:
        newDF[col] = newDF.apply(lambda _: '', axis=1)
        
    for row in dfmatches.iterrows():
        indexA = row[0][0]
        indexB = row[0][1]
        
        for col in columns:
            newDF.loc[indexA][col] = dfB.loc[indexB][col]
    return newDF

In [ ]:
result = fuse(grants, inventor, matches)
result.head(10)

### Fellegi Sunter

Now let's do this with a machine learning classifier. Keep in mind that most classifiers can not handle comparison vectors with missing values. To prevent issues with the classification algorithms. Supervised learning algorithms do need training data. Training data is data for which the true match status is known for each comparison vector. In the example in this section, we consider that the true match status of the first 5000 record pairs of our data is known.

In [ ]:
## Generate Training Data and index
ml_pairs = matches[0:40000]
ml_matches_index = ml_pairs.index & pairs2 

The Naive Bayes classifier is a probabilistic classifier. The probabilistic record linkage framework by Fellegi and Sunter (1969) is the most well-known probabilistic classification method for record linkage. Later, it was proved that the Fellegi and Sunter method is mathematically equivalent to the Naive Bayes method in case of assuming independence between comparison variables.

In [ ]:
## Train the classifier
nb = rl.NaiveBayesClassifier()
nb.learn(ml_pairs, ml_matches_index)

## Predict the match status for all record pairs
result_nb = nb.predict(matches)

## Predict probability for record to be a match
prob_nb = nb.prob(matches)## Check header

In [ ]:
## Check header
prob_nb

### Evaluation

The last step is to evaluate the results of the record linkage. We will cover this in more detail in the machine learning session. This is just for completeness.

In [ ]:
## Confusion matrix
conf_nb = rl.confusion_matrix(pairs2, result_nb, len(matches))
conf_nb

In [ ]:
## Precision and Accuracy
precision = rl.precision(conf_nb)
accuracy = rl.accuracy(conf_nb)

In [ ]:
## Precision and Accuracy
print(precision)
print(accuracy)

In [ ]:
## The F-score for this classification is
rl.fscore(conf_nb)

## References and Further Readings


### Parsing

* Python online documentation: https://docs.python.org/2/library/string.html#deprecated-string-functions
* Python 2.7 Tutorial(Splitting and Joining Strings): http://www.pitt.edu/~naraehan/python2/split_join.html

### Regular Expression

* Python documentation: https://docs.python.org/2/library/re.html#regular-expression-syntax
* Online regular expression tester (good for learning): http://regex101.com/

### String Comparators

* GitHub page of jellyfish: https://github.com/jamesturk/jellyfish
* Different distances that measure the differences between strings:
    - Levenshtein distance: https://en.wikipedia.org/wiki/Levenshtein_distance
    - Damerau–Levenshtein distance: https://en.wikipedia.org/wiki/Damerau%E2%80%93Levenshtein_distance
    - Jaro–Winkler distance: https://en.wikipedia.org/wiki/Jaro%E2%80%93Winkler_distance
    - Hamming distance: https://en.wikipedia.org/wiki/Hamming_distance
    - Match rating approach: https://en.wikipedia.org/wiki/Match_rating_approach

### Fellegi-Sunter Record Linkage 

* Introduction to Probabilistic Record Linkage: http://www.bristol.ac.uk/media-library/sites/cmm/migrated/documents/problinkage.pdf
* Paper Review: https://www.cs.umd.edu/class/spring2012/cmsc828L/Papers/HerzogEtWires10.pdf

